In [1]:
%who

Interactive namespace is empty.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings("ignore")

/Users/hanlinzhuang/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
data = pd.read_csv('../../raw data/new/eth_hourly.csv')

In [4]:
data['vol_per_count'] = data['volume_traded']/data['trades_count']
data['roi'] = np.log(data['price_close']/data['price_close'].shift())*100
data['price_close_lag1'] = data['price_close'].shift(1)
data['price_close_lag2'] = data['price_close'].shift(2)
data['price_close_lag3'] = data['price_close'].shift(3)
data['volume_traded_lag1'] = data['volume_traded'].shift(1)
data['trades_count_lag1'] = data['trades_count'].shift(1)
data['vol_per_count_lag1'] = data['vol_per_count'].shift(1)
data['roi_lag1'] = data['roi'].shift(1)
data['roi_lag2'] = data['roi'].shift(2)
data['roi_lag3'] = data['roi'].shift(3)

In [5]:
data['roi'].describe()

count    2135.000000
mean        0.022361
std         2.050317
min       -12.726184
25%        -0.575787
50%        -0.003578
75%         0.618430
max        10.254219
Name: roi, dtype: float64

In [6]:
std = data['roi'].std()
data['cut'] = pd.cut(data['roi'], 
                     bins=[-100, -3*std, -2*std, -std, 0, std, 2*std, 3*std, 100], 
                    labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4'])
data['cut_lag1'] = data['cut'].shift(1)
data['cut_lag2'] = data['cut'].shift(2)
data['cut_lag3'] = data['cut'].shift(3)

In [7]:
data.head()

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count,...,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1,roi_lag1,roi_lag2,roi_lag3,cut,cut_lag1,cut_lag2,cut_lag3
0,2019-02-01T00:00:00.0000000Z,2019-02-01T01:00:00.0000000Z,2019-02-01T00:00:00.0000000Z,2019-02-01T00:59:56.8440000Z,105.84,115.898980,103.34642,105.250000,15306.01465,2118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-02-01T01:00:00.0000000Z,2019-02-01T02:00:00.0000000Z,2019-02-01T01:00:01.4630000Z,2019-02-01T01:59:59.3180000Z,105.31,115.010000,101.56001,103.260000,76449.76249,8277,...,15306.01465,2118.0,7.226636,NaN,NaN,NaN,-1,NaN,NaN,NaN
2,2019-02-01T02:00:00.0000000Z,2019-02-01T03:00:00.0000000Z,2019-02-01T02:00:00.1500000Z,2019-02-01T02:59:51.1550000Z,103.23,115.000000,101.30185,103.730000,24897.83393,3262,...,76449.76249,8277.0,9.236410,-1.908839,NaN,NaN,+1,-1,NaN,NaN
3,2019-02-01T03:00:00.0000000Z,2019-02-01T04:00:00.0000000Z,2019-02-01T03:00:03.0000000Z,2019-02-01T03:59:58.9570000Z,106.38,114.908412,103.66946,104.200000,13256.61415,2030,...,24897.83393,3262.0,7.632690,0.454129,-1.908839,NaN,+1,+1,-1,NaN
4,2019-02-01T04:00:00.0000000Z,2019-02-01T05:00:00.0000000Z,2019-02-01T04:00:00.4540000Z,2019-02-01T04:59:57.0000000Z,104.08,115.499880,103.33000,113.846545,10376.78714,2148,...,13256.61415,2030.0,6.530352,0.452076,0.454129,-1.908839,+4,+1,+1,-1


In [8]:
data.columns

Index(['time_period_start', 'time_period_end', 'time_open', 'time_close',
       'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded',
       'trades_count', 'vol_per_count', 'roi', 'price_close_lag1',
       'price_close_lag2', 'price_close_lag3', 'volume_traded_lag1',
       'trades_count_lag1', 'vol_per_count_lag1', 'roi_lag1', 'roi_lag2',
       'roi_lag3', 'cut', 'cut_lag1', 'cut_lag2', 'cut_lag3'],
      dtype='object')

In [9]:
data_ = data.dropna()
data_.head()

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count,...,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1,roi_lag1,roi_lag2,roi_lag3,cut,cut_lag1,cut_lag2,cut_lag3
4,2019-02-01T04:00:00.0000000Z,2019-02-01T05:00:00.0000000Z,2019-02-01T04:00:00.4540000Z,2019-02-01T04:59:57.0000000Z,104.08000,115.499880,103.33000,113.846545,10376.787140,2148,...,13256.614150,2030.0,6.530352,0.452076,0.454129,-1.908839,+4,+1,+1,-1
5,2019-02-01T05:00:00.0000000Z,2019-02-01T06:00:00.0000000Z,2019-02-01T05:00:03.0000000Z,2019-02-01T05:59:51.1920000Z,106.50000,114.000000,102.15319,103.940000,6840.788744,1669,...,10376.787140,2148.0,4.830906,8.853931,0.452076,0.454129,-4,+4,+1,+1
6,2019-02-01T06:00:00.0000000Z,2019-02-01T07:00:00.0000000Z,2019-02-01T06:00:00.0000000Z,2019-02-01T06:59:59.0000000Z,106.72000,114.699990,101.49483,104.190000,7856.088726,1585,...,6840.788744,1669.0,4.098735,-9.103763,8.853931,0.452076,+1,-4,+4,+1
7,2019-02-01T07:00:00.0000000Z,2019-02-01T08:00:00.0000000Z,2019-02-01T07:00:00.0000000Z,2019-02-01T07:59:58.0000000Z,106.92000,114.531990,101.30200,106.150000,30112.397610,3842,...,7856.088726,1585.0,4.956523,0.240235,-9.103763,8.853931,+1,+1,-4,+4
8,2019-02-01T08:00:00.0000000Z,2019-02-01T09:00:00.0000000Z,2019-02-01T08:00:00.0000000Z,2019-02-01T08:59:56.6300000Z,106.14144,114.531994,101.24071,104.540000,32759.219620,3039,...,30112.397610,3842.0,7.837688,1.863703,0.240235,-9.103763,-1,+1,+1,-4


In [10]:
y = data_['price_close']
X = data_[['price_close_lag1', 'price_close_lag2', 'price_close_lag3', 
           'roi_lag1', 
           'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [12]:
mod_linear = LinearRegression()
mod_linear.fit(X_train, y_train);

In [13]:
print('coefficients:')
for i in range(len(X.columns)):
    print('{}: {}'.format(X.columns[i], mod_linear.coef_[i]))

coefficients:
price_close_lag1: 0.9785345125006797
price_close_lag2: -0.2068197459178823
price_close_lag3: 0.22250272430295054
roi_lag1: -0.6775798647002288
volume_traded_lag1: 1.2551435292362411e-05
trades_count_lag1: -0.00010317279630834032
vol_per_count_lag1: -0.10344943405607836


In [14]:
y_predicted = mod_linear.predict(X_test)
print('mse:')
print(mean_squared_error(y_test, y_predicted))

mse:
6.331828002729407


In [15]:
print('accuracy:')
print(mod_linear.score(X_test, y_test))

accuracy:
0.8736404265966291


### clf

In [16]:
# y_clf = data_['cut']
# X_clf = data_[['roi_lag1', 'roi_lag2', 'roi_lag3', 
#                'price_close_lag1', 
#                'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [17]:
y_clf = data_['cut']
X_clf = data_[['roi_lag1',  
               'price_close_lag1', 
               'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [18]:
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=0)

In [19]:
# random forest

In [20]:
clf_rf = RandomForestClassifier(n_estimators=1000, max_depth=5, bootstrap=True, random_state=0)
clf_rf.fit(X_clf_train, y_clf_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [21]:
print('accuracy:')
print(clf_rf.score(X_clf_test, y_clf_test))

accuracy:
0.4613583138173302


In [22]:
y_clf_predicted = clf_rf.predict(X_clf_test)
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted, labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4']), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,6,0,0,0,1,0,0,0
Actual -3,1,2,1,0,1,0,0,0
Actual -2,0,0,18,15,11,0,0,0
Actual -1,0,0,4,71,86,0,0,0
Actual +1,0,0,7,49,100,0,0,0
Actual +2,0,0,0,12,32,0,0,0
Actual +3,0,0,1,0,6,0,0,0
Actual +4,0,0,0,1,2,0,0,0


In [23]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted, average = None))

Average precision for the 8 intervals is -  [0.41841004 0.         0.         0.         0.47972973 0.58064516
 1.         0.85714286]


In [24]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted, average = None))

Average recall for the 8 intervals is -  [0.64102564 0.         0.         0.         0.44099379 0.40909091
 0.4        0.85714286]


In [25]:
# boosting

In [26]:
clf_gb = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, 
                                            max_depth=1, random_state=0)
clf_gb.fit(X_clf_train, y_clf_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)

In [27]:
print('accuracy:')
print(clf_gb.score(X_clf_test, y_clf_test))

accuracy:
0.207444333345001


In [28]:
y_clf_predicted = clf_gb.predict(X_clf_test)
y_clf_predicted = pd.DataFrame(y_clf_predicted)
y_clf_predicted['cut'] = pd.cut(y_clf_predicted[0], 
                     bins=[-100, -3*std, -2*std, -std, 0, std, 2*std, 3*std, 100], 
                    labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4'])

In [29]:
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted['cut'], labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4']), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,0,0,6,0,1,0,0,0
Actual -3,0,0,3,1,1,0,0,0
Actual -2,0,0,0,31,13,0,0,0
Actual -1,0,0,0,47,114,0,0,0
Actual +1,0,0,0,38,118,0,0,0
Actual +2,0,0,0,8,36,0,0,0
Actual +3,0,0,0,2,5,0,0,0
Actual +4,0,0,0,0,3,0,0,0


In [30]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted['cut'], average = None))

Average precision for the 8 intervals is -  [0.40549828 0.         0.         0.         0.37007874 0.
 0.         0.        ]


In [31]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted['cut'], average = None))

Average recall for the 8 intervals is -  [0.75641026 0.         0.         0.         0.29192547 0.
 0.         0.        ]


In [32]:
# Adaboosting

In [33]:
clf_adab = AdaBoostClassifier(n_estimators=1000, learning_rate=1, algorithm='SAMME')
clf_adab.fit(X_clf_train,y_clf_train)

AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=1,
          n_estimators=1000, random_state=None)

In [34]:
print('accuracy:')
print(clf_adab.score(X_clf_test, y_clf_test))

accuracy:
0.3911007025761124


In [35]:
y_clf_predicted = clf_adab.predict(X_clf_test)
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted, labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4']), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,1,2,1,2,1,0,0,0
Actual -3,0,3,0,1,1,0,0,0
Actual -2,0,4,9,23,8,0,0,0
Actual -1,0,1,4,51,98,5,2,0
Actual +1,0,0,6,41,102,7,0,0
Actual +2,0,0,1,9,32,0,2,0
Actual +3,0,0,0,4,2,0,1,0
Actual +4,0,0,0,0,3,0,0,0


In [36]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted, average = None))

Average precision for the 8 intervals is -  [0.41295547 0.         0.2        0.         0.38931298 0.42857143
 0.3        1.        ]


In [37]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted, average = None))

Average recall for the 8 intervals is -  [0.65384615 0.         0.14285714 0.         0.31677019 0.20454545
 0.6        0.14285714]
